In [17]:
import psycopg2
from zipfile import ZipFile
import pandas as pd
import os
from io import StringIO
import numpy as np
import shutil

zipped_folder=os.path.join('C:\\Users\\Munira\\Desktop\\demo-pipeline','zipped_raw_data')

unzipped_folder=os.path.join('C:\\Users\\Munira\\Desktop\\demo-pipeline','raw_data')

def connect_to_db(db_name,password):
    conn = psycopg2.connect(f"dbname={db_name} user=dataengineer host='mydb.cb7sygkkbh0u.us-east-2.rds.amazonaws.com' port=5432 password={password}")
    cur = conn.cursor()
    return conn,cur

def extract_and_unzip_files(zipped_folder, unzipped_folder):
    for zip_filename in os.listdir(zipped_folder):
        z=ZipFile(os.path.join(zipped_folder,zip_filename))
        z.extractall(unzipped_folder)



In [25]:
def move_files_to_new_subfolder(new_folder_name,file_identifier,original_file_location):
    try:
        # Create subfolder
        os.mkdir(os.path.join(unzipped_folder,new_folder_name))
        # Get all files with file_identifier as part of their filenames
        files_list=[x  for x in os.listdir(unzipped_folder) if file_identifier in x]
        # Move files to new subfolder
        for file in files_list:
            shutil.move(os.path.join(original_file_location,file), os.path.join(original_file_location,new_folder_name,file))
    except FileExistsError:
        pass
    print('Done')

In [27]:
extract_and_unzip_files(zipped_folder,unzipped_folder)

move_files_to_new_subfolder('ReadMeFiles','Readme',unzipped_folder)

move_files_to_new_subfolder('DataFiles','txt',unzipped_folder)

### Creating tables

In [ ]:
CREATE TABLE crash (
  Report_number VARCHAR(50) NOT NULL,
  Report_seq_no REAL NOT NULL,
  DOT_Number VARCHAR(50),
  Report_Date DATE,
  Report_State CHAR(2),
  Fatalities REAL,
  Injuries REAL,
  Tow_Away CHAR(1),
  Hazmat_released CHAR(1),
  Trafficway_Desc VARCHAR(255),
  Access_Control_Desc VARCHAR(255),
  Road_surface_Condition_Desc VARCHAR(255),
  Weather_Condition_Desc VARCHAR(255),
  Light_Condition_Desc VARCHAR(255),
  Vehicle_ID_Number VARCHAR(50),
  Vehicle_License_number VARCHAR(50),
  Vehicle_license_state CHAR(2),
  Severity_Weight REAL,
  Time_weight REAL,
  citation_issued_desc VARCHAR(255),
  seq_num REAL,
  Not_Preventable CHAR(1)
);


### Copying data into postgres

In [81]:
low_cols=[x.lower() for x in list(df.columns)]

In [85]:
connection,cursor=connect_to_db('staging','aculocity')

In [86]:
cursor.copy_from?

In [82]:
def insert_csv(f, table, columns):
    connection,cursor=connect_to_db('staging','aculocity')
    try:
        cursor.copy_from(f, table, sep=',', columns=columns,null='')
        connection.commit()
        return True
    except (psycopg2.Error) as e:
        print(e)
        return False
    finally:
        cursor.close()
        connection.close()
        


In [83]:
filepath=os.path.join(unzipped_folder,'CleanedData','2022Aug_Crash.csv')

In [84]:
with open(filepath) as file:
    insert_csv(file, "crash", low_cols)

invalid input syntax for type real: "REPORT_SEQ_NO"
CONTEXT:  COPY crash, line 1, column report_seq_no: "REPORT_SEQ_NO"



## Modifying raw data files

In [60]:
os.mkdir(os.path.join(unzipped_folder,'CleanedData'))

In [90]:
os.path.join(unzipped_folder,'CleanedData').split('\\')[-1]

'CleanedData'

In [106]:
def clean_raw_data_files(original_filepath,cleaned_folder_path):

    df=pd.read_csv(original_filepath,encoding='cp1252')

    # Remove rows where all values are missing
    df=df.dropna(how='all')

    # Replacing empty strings for float columns with nans
    num_cols = [col for col in df.columns if df[col].dtype in ['float64','int64']]
    for col in num_cols:
        df[col]=df[col].fillna(np.nan)
        df.loc[df[col]=='',col]=np.nan

    # Replacing nulls with empty strings for text columns
    cols=[col for col in df.columns if df[col].dtype=='O']
    for col in cols:
        df[col]=df[col].fillna('')

    # Write into CSV file
    df.to_csv(os.path.join(cleaned_folder_path,os.path.split(original_filepath)[-1].replace('.txt','.csv')),index=False)


In [107]:
original_filepath=os.path.join(unzipped_folder,'DataFiles','2022Aug_Inspection.txt')

In [108]:
cleaned_folder_path=os.path.join(unzipped_folder,'CleanedData')

In [ ]:
clean_raw_data_files(original_filepath,cleaned_folder_path)

In [102]:
original_filepath=os.path.join(unzipped_folder,'DataFiles','2022Aug_Inspection.txt')

In [104]:
df=pd.read_csv(original_filepath,encoding='cp1252')

C:\Users\Munira\AppData\Local\Temp\ipykernel_16884\798137644.py:1: DtypeWarning: Columns (19,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(original_filepath,encoding='cp1252')


In [105]:
df

,UNIQUE_ID,REPORT_NUMBER,REPORT_STATE,DOT_NUMBER,INSP_DATE,INSP_LEVEL_ID,COUNTY_CODE_STATE,TIME_WEIGHT,DRIVER_OOS_TOTAL,VEHICLE_OOS_TOTAL,...,SUBT_ALCOHOL_INSP,VH_MAINT_INSP,HM_INSP,BASIC_VIOL,UNSAFE_VIOL,FATIGUED_VIOL,DR_FITNESS_VIOL,SUBT_ALCOHOL_VIOL,VH_MAINT_VIOL,HM_VIOL
0,635384790,3019002064,CT,2582962,26-AUG-20,2,CT,1,0,3,...,Y,Y,NaN,5,1,0,0,0,4,0.0
1,635384799,3021005363,CT,1474945,26-AUG-20,5,CT,1,0,0,...,NaN,Y,NaN,0,0,0,0,0,0,0.0
2,635384808,3021005364,CT,1474945,26-AUG-20,5,CT,1,0,0,...,NaN,Y,NaN,0,0,0,0,0,0,0.0
3,635384817,3078000110,CT,817929,26-AUG-20,1,CT,1,0,0,...,Y,Y,NaN,0,0,0,0,0,0,0.0
4,635384826,3078000111,CT,601324,26-AUG-20,1,CT,1,0,0,...,Y,Y,NaN,1,0,0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5493696,687977028,UEX8002323,CA,2851454,23-AUG-22,1,CA,3,0,0,...,Y,Y,NaN,2,0,0,0,0,2,0.0
5493697,687977037,UFJT000674,CA,763935,25-AUG-22,1,CA,3,0,0,...,Y,Y,NaN,0,0,0,0,0,0,0.0
5493698,687977046,UFJT000675,CA,3167943,25-AUG-22,1,CA,3,0,1,...,Y,Y,NaN,1,0,0,0,0,1,0.0
5493699,687977055,UFJT000676,CA,2626512,25-AUG-22,1,CA,3,0,3,...,Y,Y,NaN,7,0,0,0,0,7,0.0


#### Removing rows where all values are null/missing

In [46]:
df.shape

(281285, 22)

In [49]:
df=df.dropna(how='all')

In [50]:
df.shape[0] - df.dropna(how='all').shape[0]

0

In [51]:
df=df.dropna(how='all')

### Float columns - dealing with nulls showing up as empty strings

In [52]:
df.loc[df['REPORT_SEQ_NO']=='','REPORT_SEQ_NO']=np.nan

In [53]:
df.loc[df['REPORT_SEQ_NO'].isnull(),'REPORT_SEQ_NO']=np.nan

In [71]:
num_cols = [col for col in df.columns if df[col].dtype in ['float64','int64']]
for col in num_cols:
    df[col]=df[col].fillna(np.nan)

In [72]:
num_cols

['REPORT_SEQ_NO',
 'DOT_NUMBER',
 'FATALITIES',
 'INJURIES',
 'SEVERITY_WEIGHT',
 'TIME_WEIGHT',
 'SEQ_NUM']

In [77]:
df[df['SEVERITY_WEIGHT'].isnull()]

,REPORT_NUMBER,REPORT_SEQ_NO,DOT_NUMBER,REPORT_DATE,REPORT_STATE,FATALITIES,INJURIES,TOW_AWAY,HAZMAT_RELEASED,TRAFFICWAY_DESC,...,WEATHER_CONDITION_DESC,LIGHT_CONDITION_DESC,VEHICLE_ID_NUMBER,VEHICLE_LICENSE_NUMBER,VEHICLE_LICENSE_STATE,SEVERITY_WEIGHT,TIME_WEIGHT,CITATION_ISSUED_DESC,SEQ_NUM,NOT_PREVENTABLE
305,AL0000695573,2.0,3226506.0,27-SEP-20,AL,1.0,1.0,Y,N,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Daylight,CA213HM127039,1143165,AL,NaN,NaN,NO,2.0,
425,AL0000699975,2.0,165182.0,07-OCT-20,AL,0.0,0.0,Y,N,Two-Way Trafficway Not Divided,...,No Adverse Conditions,Daylight,1HTMMMMN6JH421178,MX5960,NC,NaN,NaN,NO,2.0,
966,AL0000721414,1.0,2002537.0,24-NOV-20,AL,0.0,0.0,Y,,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Daylight,1XKYD49X1GJ477835,IRL4086,NM,NaN,NaN,NO,2.0,
996,AL0000722923,2.0,835435.0,02-DEC-20,AL,0.0,0.0,Y,N,Two-Way Trafficway Not Divided,...,No Adverse Conditions,Daylight,2HSFHAMR5YC051037,,UK,NaN,NaN,NO,2.0,
1553,AL0001610528,2.0,92366.0,28-JAN-21,AL,0.0,0.0,Y,N,Two-Way Trafficway Divided Unprotected Median,...,No Adverse Conditions,Daylight,YE2XC21B5F3048673,1122037,AL,NaN,NaN,NO,2.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277760,WV202105088A,3.0,467572.0,02-MAR-21,WV,0.0,0.0,Y,,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Daylight,1XPBD49X3GD330334,BA542046,WV,NaN,NaN,UNKNOWN,2.0,
277929,WV202111055A,2.0,2579293.0,07-MAY-21,WV,0.0,0.0,Y,N,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Daylight,1XKWD49X1LR345516,P767175,SC,NaN,NaN,UNKNOWN,2.0,
278260,WV202122952A,2.0,3098005.0,15-SEP-21,WV,0.0,1.0,Y,N,Two-Way Trafficway Not Divided,...,Rain,Daylight,1XPBA48XXED199250,A451020,MS,NaN,NaN,UNKNOWN,2.0,
278284,WV202123775A,2.0,3486064.0,23-SEP-21,WV,0.0,0.0,Y,N,Two-Way Trafficway Divided Positive Barrier,...,Rain,Daylight,4V4NC9EHXNN295817,XKE714,GA,NaN,NaN,UNKNOWN,2.0,


In [76]:
df['SEVERITY_WEIGHT'].unique()

array([ 1.,  2., nan,  3.])

In [69]:
df['SEVERITY_WEIGHT'].dtype=='float64'

True

In [64]:
df['SEVERITY_WEIGHT']=df['SEVERITY_WEIGHT'].fillna(np.nan)

#### Text columns - filling in nulls with empty strings

In [54]:
cols=[col for col in df.columns if df[col].dtype=='O']

In [55]:
cols

['REPORT_NUMBER',
 'REPORT_DATE',
 'REPORT_STATE',
 'TOW_AWAY',
 'HAZMAT_RELEASED',
 'TRAFFICWAY_DESC',
 'ACCESS_CONTROL_DESC',
 'ROAD_SURFACE_CONDITION_DESC',
 'WEATHER_CONDITION_DESC',
 'LIGHT_CONDITION_DESC',
 'VEHICLE_ID_NUMBER',
 'VEHICLE_LICENSE_NUMBER',
 'VEHICLE_LICENSE_STATE',
 'CITATION_ISSUED_DESC',
 'NOT_PREVENTABLE']

In [56]:
for col in cols:
    df[col]=df[col].fillna('')

In [57]:
df

,REPORT_NUMBER,REPORT_SEQ_NO,DOT_NUMBER,REPORT_DATE,REPORT_STATE,FATALITIES,INJURIES,TOW_AWAY,HAZMAT_RELEASED,TRAFFICWAY_DESC,...,WEATHER_CONDITION_DESC,LIGHT_CONDITION_DESC,VEHICLE_ID_NUMBER,VEHICLE_LICENSE_NUMBER,VEHICLE_LICENSE_STATE,SEVERITY_WEIGHT,TIME_WEIGHT,CITATION_ISSUED_DESC,SEQ_NUM,NOT_PREVENTABLE
0,AK0020039340,1.0,8294,15-DEC-20,AK,0,0,Y,N,One-Way Trafficway Not Divided,...,Snow,Dark - Lighted,2XKHDJ9X1GM481794,GXX560,AK,1.0,1.0,UNKNOWN,1,
1,AK0020039542,1.0,190356,17-DEC-20,AK,0,0,Y,,One-Way Trafficway Not Divided,...,No Adverse Conditions,Daylight,1XKZD49X4KJ303397,KCX725,AK,1.0,1.0,YES,1,
2,AK0020039828,1.0,3120623,19-DEC-20,AK,0,1,Y,,Two-Way Trafficway Divided Positive Barrier,...,Blowing Sand Soil Dirt Or Snow,Dark - Lighted,1HTMMMML5JH568106,TOWAK,AK,2.0,1.0,UNKNOWN,1,
3,AK0020081042,1.0,1640646,27-OCT-20,AK,1,1,Y,,Two-Way Trafficway Not Divided,...,Blowing Sand Soil Dirt Or Snow,Dark - Not Lighted,2NP3LJ0X9EM239931,GSA386,AK,2.0,1.0,UNKNOWN,1,
4,AK0020095165,1.0,248628,08-DEC-20,AK,1,3,Y,,Two-Way Trafficway Not Divided,...,No Adverse Conditions,Daylight,J8DC4W16187003451,FPB425,AK,2.0,1.0,UNKNOWN,1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281280,WY0202209128,1.0,3749576,21-AUG-22,WY,0,0,Y,,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Daylight,1XKYD49X3NJ101005,YP57687,CA,1.0,3.0,NO,1,
281281,WY0202209132,1.0,53467,09-AUG-22,WY,0,0,Y,,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Dark - Not Lighted,3HSDZAPR8MN683541,W33995,NE,1.0,3.0,NO,1,
281282,WY0202209156,1.0,107540,16-AUG-22,WY,0,0,Y,,Two-Way Trafficway Divided Unprotected Median,...,No Adverse Conditions,Daylight,1NPTL40X8DD200284,2SG524,OK,1.0,3.0,NO,1,
281283,WY0502207230,1.0,524401,10-JUL-22,WY,0,0,Y,,Two-Way Trafficway Not Divided,...,No Adverse Conditions,Dark - Not Lighted,WCDPF1CD0KT00324,KAF173,ID,1.0,3.0,NO,1,


In [58]:
df.to_csv(os.path.join(unzipped_folder,'2022Aug_Crash.csv'),index=False)

## Copying into postgres

In [ ]:
\copy crash FROM 'C:\\Users\Munira\Desktop\demo-pipeline\raw_data\CleanedData\2022Aug_Crash.csv' WITH (FORMAT CSV, DELIMITER ',',HEADER);